# VGG16 from scratch for my dogscats classifier
This is possible from MOOC course www.fast.ai
Let's have a look at Vgg16 architecture.

In [1]:
import vgg16

Using TensorFlow backend.


In [2]:
from vgg16 import VGG16
vgg = VGG16()

In [3]:
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

Lets Construct the same architecture

In [4]:
import keras
keras.__version__

'2.0.5'

In [5]:
import tensorflow
tensorflow.__version__

'1.2.1'

Vgg16 has:
1. 2 convolution layers with zero padding layer with maxpooling layer
2. 2 convolution layers with zero padding layer with maxpooling layer
3. 3 convolution layers with zero padding layer with maxpooling layer
4. 3 convolution layers with zero padding layer with maxpooling layer
5. 3 convolution layers with zero padding layer with maxpooling layer
6. With flattening layer

Vgg16 has 3 Dense layers. 
1. 1st 2 dense layer with 4096 neurons. 
2. 3rd dense layer cotains 1000 neurons (because it has to classify 1000 classes). 
3. It has also dropout layer with value of 0.5. 

# Dropout layer
Dropout is used to reduce overfitting. It deletes weigths of certian neurons ( a.k.a deactivating neurons ). Typically it's value is 50% (so 0.5). Some engineers use 0.25, 0.35, 0.45 respectively for each dense layer they add.  

In [6]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Dense, Dropout
from keras.layers import Flatten

#This will be a sequential model
scratch = Sequential()

#1
scratch.add(Convolution2D( 64, 3, 3, input_shape = (224,224,3), activation='relu') )
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Convolution2D(64,3,3, activation='relu') )
scratch.add(MaxPooling2D(pool_size=(2,2)) )

#2
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Convolution2D(128,3,3,  activation='relu') )
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Convolution2D(128,3,3,  activation='relu') )
scratch.add(MaxPooling2D(pool_size=(2,2)) )

#3
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Convolution2D(256,3,3,  activation='relu') )
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Convolution2D(256,3,3,  activation='relu') )
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Convolution2D(256,3,3,  activation='relu') )
scratch.add(MaxPooling2D(pool_size=(2,2) ))

#3
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Convolution2D(512,3,3,  activation='relu') )
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Convolution2D(512,3,3,  activation='relu') )
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Convolution2D(512,3,3,  activation='relu') )
scratch.add(MaxPooling2D(pool_size=(2,2)))

#3
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Convolution2D(512,3,3,  activation='relu') )
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Convolution2D(512,3,3,  activation='relu') )
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Convolution2D(512,3,3,  activation='relu') )
scratch.add(MaxPooling2D(pool_size=(2,2) ))

#Flatten ing layer
scratch.add(Flatten())

#Dense 1
scratch.add(Dense(4096,activation='relu'))
scratch.add(Dropout(0.5))    

#Dense 2
scratch.add(Dense(4096,activation='relu'))
scratch.add(Dropout(0.5))    

#Finetuning
scratch.add(Dense(1,activation='sigmoid'))    

C:\Users\Chaitra\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", input_shape=(224, 224,...)`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Chaitra\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  if sys.path[0] == '':
C:\Users\Chaitra\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
C:\Users\Chaitra\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
C:\Users\Chaitra\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
C:\Users\Chaitra\Anaconda3\lib\site-packag

In [7]:
scratch.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator()


train_generator = gen.flow_from_directory(
        'training_set',
        target_size=(224, 224),
        batch_size=3,
        class_mode='binary')

test_generator = gen.flow_from_directory(
        'test_set',
        target_size=(224, 224),
        batch_size=2,
        class_mode='binary')
#According to vgg16 model, target_size must be 224,224. I have given target_size as 64,64 to avoid errors. 
# I will figure it out and update. 
scratch.fit_generator(
        train_generator,
        steps_per_epoch=3,
        epochs=5,
        validation_data=test_generator,
        validation_steps=2)

Found 20 images belonging to 2 classes.
Found 6 images belonging to 2 classes.
Epoch 1/5


ResourceExhaustedError: OOM when allocating tensor with shape[4096,4096]
	 [[Node: dense_2/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=7200915, _device="/job:localhost/replica:0/task:0/gpu:0"](dense_2/random_uniform/shape)]]

Caused by op 'dense_2/random_uniform/RandomUniform', defined at:
  File "C:\Users\Chaitra\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Chaitra\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Chaitra\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Chaitra\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Chaitra\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\Chaitra\Anaconda3\lib\site-packages\zmq

## I dont know what exactly this error means. So, left here . 

In [ ]:
from keras.models import load_model
scratch.save('vgg_scratch.h5')